In [3]:
import pandas as pd
import datasets
from datasets import load_dataset, Dataset, DatasetDict

Qdata_frame = pd.read_csv('../dataset/train1.csv')
# Qdf_train = Qdata_frame[0:275]
# Qdf_test = Qdata_frame[275:]

# trainDS = Dataset.from_pandas(Qdf_train)
# testDS = Dataset.from_pandas(Qdf_test)
# ds = DatasetDict()

# ds["train"] = trainDS
# ds["test"]  = testDS

In [5]:
# split the train, test, and validation data
import sklearn
from sklearn.model_selection import train_test_split

Qdf_train, Qdf_remain = train_test_split(Qdata_frame, train_size=0.8)
Qdf_valid, Qdf_test = train_test_split(Qdf_remain, test_size=0.5)

In [21]:
Qdf_train.head(2)

,id,String,topic,type
318,319,Ada is going to take photos of the lunch sets ...,NaN,LQ
221,222,How many students are there in total?,"Algorithm Design, Program Development",LQ


In [22]:
Qdf_valid.head(2)

,id,String,topic,type
172,173,Which of the following should Clara do to incr...,Internet Services and Applications,MC
260,261,Mark watches live stream on this platform for ...,Health and Ethical Issues,LQ


In [23]:
Qdf_test.head(2)

,id,String,topic,type
239,240,Which of the following are methods of authenti...,Threats and Security on the Internet,MC
58,59,Give an advantage of using JPG over BMP.,Multimedia Elements,LQ


In [20]:
# change to hugging face dataset type
trainDS = Dataset.from_pandas(Qdf_train)
validDS = Dataset.from_pandas(Qdf_valid)
testDS = Dataset.from_pandas(Qdf_test)
ds = DatasetDict()
ds["train"] = trainDS
ds["valid"] = validDS
ds["test"] = testDS
print(ds)


Dataset({
    features: ['id', 'String', 'topic', 'type', '__index_level_0__'],
    num_rows: 341
})

In [25]:
#  clear not used columns
columns = ds.column_names
columns_toRemove = ["id", "type"]
new_ds = ds.remove_columns(columns_toRemove)
print(new_ds)


DatasetDict({
    train: Dataset({
        features: ['String', 'topic', '__index_level_0__'],
        num_rows: 341
    })
    valid: Dataset({
        features: ['String', 'topic', '__index_level_0__'],
        num_rows: 43
    })
    test: Dataset({
        features: ['String', 'topic', '__index_level_0__'],
        num_rows: 43
    })
})


In [ ]:
# TODO: tokenize text, feature extraction, multilabel classification
 